# Durham Transmission Calculation for Compressed PSFs
This notebook is used to compare the compressed and uncompressed Durham PSFs, which are needed to speed up the transmission calculations

In [1]:
from astropy.io import fits
from astropy import units as u
import numpy as np
import math
from astropy.modeling.functional_models import Disk2D
import matplotlib.pyplot as plt
plt.style.use('bmh')
import time

In [2]:
def numerical_durham_speed_v3(diameter,wavelength,offset,axis_val=24,version=0):
    file=fits.open("PSFs/GLAO_Median_{}nm_v2.fits".format(round(wavelength.value)))
    durham_data=file[version].data[axis_val]
    scale=file[version].header['scale']

    fibre_boundary=math.ceil(diameter.value/2/scale)
    data_boundary=len(durham_data)

    x = np.arange(-fibre_boundary,fibre_boundary+1)
    y = np.arange(-fibre_boundary, fibre_boundary+1)
    x, y = np.meshgrid(x, y)

    offset = abs(offset)
    disk=Disk2D(1,abs(int(offset.value/scale)-offset.value/scale),0,diameter.value/2/scale)
    disk_data=disk(x,y)

    resized_data=np.zeros([len(disk_data),len(disk_data)])

    durham_data=durham_data[int(data_boundary/2-fibre_boundary):int(data_boundary/2+fibre_boundary)+1,int(data_boundary/2-fibre_boundary+offset.value/scale):int(data_boundary/2+fibre_boundary+offset.value/scale)+1]
    resized_data[0:len(durham_data),0:len(durham_data[0])]=durham_data

    convolved=resized_data*disk_data
    trans=sum(sum(convolved))
    return trans

## Speed Comparison

In [3]:
diameters=np.arange(.4,.8,.01)*u.arcsec
offset=np.arange(0,0.7,0.01)*u.arcsec
wavelengths=[440,562,720,920,1202,1638]*u.micron
axis_val=24

In [4]:
start=time.time()
for wavelength in wavelengths:
    for d in diameters:
        for o in offset:
            numerical_durham_speed_v3(d*u.arcsec,wavelength,o,axis_val) #small mesh#2 neat
print("Uncompressed Time:")
print(round(time.time()-start,2))
    
start=time.time()
for wavelength in wavelengths:
    for d in diameters:
        for o in offset:
            numerical_durham_speed_v3(d*u.arcsec,wavelength,o,axis_val,version=1) #small mesh#2 neat
print("Compressed time:")
print(round(time.time()-start,2))

Uncompressed Time:
30.03
Compressed time:
20.1


## Accuracy Comparison

In [5]:
diameter=0.7*u.arcsec
wavelength=1638*u.micron
offset=+0.3*u.arcsec
axis_val=24

In [6]:
trans=numerical_durham_speed_v3(diameter,wavelength,offset,axis_val,version=0)*100
print("Uncompressed trans:")
print(round(trans,2))

trans=numerical_durham_speed_v3(diameter,wavelength,offset,axis_val,version=1)*100
print("Compressed trans:")
print(round(trans,2))

Uncompressed trans:
50.86
Compressed trans:
50.17


## Compression Code
Code used to compress Durham PSFs to ~0.01 arcsec/pixel

In [7]:
def half_data(data,times):
    newdata=data
    for i in range(0,times):
        new_bound=int(len(data)/2)
        newdata=np.zeros((new_bound,new_bound)) 
        for i in range(0,new_bound):
            for o in range(0,new_bound):
                newdata[i][o]=np.sum(data[i*2:i*2+2,o*2:o*2+2])
        data=newdata        
    print("Compressed {} times".format(times))
    scale=file[0].header['scale']
    new_scale=scale*2**times
    print("Scale now {:2f} arcsec/pixel".format(new_scale))

    return newdata,new_scale
wavelengths=[440,562,720,920,1202,1638]*u.micron

In [8]:
durham_datas=[]
wavelength=wavelengths[5]
axis_vals=np.arange(0,49)
for axis_val in axis_vals:
    file=fits.open("PSFs/GLAO_Median_{}nm.fits".format(round(wavelength.value)))
    durham_data=file['PRIMARY'].data[axis_val]
    scale=file[0].header['scale']
    print(scale)
    durham_datas.append(durham_data)

0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.004331565000000001
0.00433156500

In [9]:
durham_datas_halfed=[]
for array in durham_datas:
    durham_data_halfed,new_scale=half_data(array,1) #change second variable to compress differently
    durham_datas_halfed.append(durham_data_halfed)

Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.008663 arcsec/pixel
Compressed 1 times
Scale now 0.

In [10]:
hdu_2=fits.ImageHDU(durham_datas_halfed)
file.append(hdu_2)
file[1].header['scale']=new_scale
file.writeto("PSFs/GLAO_Median_{}nm_v2.fits".format(round(wavelength.value)))

OSError: File 'PSFs/GLAO_Median_1638nm_v2.fits' already exists.